In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


In [11]:
Corpus = pd.read_csv(r"D:\\NLP project\\dataset\\final\\cleandata2.csv",encoding='latin-1')
Corpus['title'].dropna(inplace=True)

In [12]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['title'],Corpus['label'],test_size=0.3,shuffle=True)

In [13]:

print(len(Train_X))
print(len(Train_Y))
print(len(Test_X))
print(len(Test_Y))

1693087
1693087
725610
725610


In [14]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y.astype(str))
Test_Y = Encoder.fit_transform(Test_Y.astype(str))


In [15]:
training_labels_final = np.array(Train_Y)
testing_labels_final = np.array(Test_Y)

In [8]:
vocab_size = 1200000
embedding_dim = 32
max_length = 220
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(Train_X)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(Train_X)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(Test_X)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [9]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 220, 32)           38400000  
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 38,453,301
Trainable params: 38,453,301
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
num_epochs = 1
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))




Train on 1693088 samples, validate on 725610 samples
Epoch 1/1





1693088/1693088 [==============================] - 40404s 24ms/step - loss: 0.0287 - acc: 0.9906 - val_loss: 0.0200 - val_acc: 0.9936


In [15]:
#saving
import pickle
with open('D:\\NLP project\\tokenizer2.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    


FileNotFoundError: [Errno 2] No such file or directory: 'tokenizer.pickle'

In [20]:
model.save('D:\\NLP project\\models\\LSTMFinalModel1.h5')

In [7]:
padding_type='post'
sentence = ["lifes too short as it is"]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
x=model.predict(padded)
print(x)
if x>0.5:
    print("inapprpriate context")
else:
    print("appropriate context")

[[0.17461154]]
appropriate context


In [10]:
import pickle 
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_length = 120
trunc_type='post'
f = open('D:/NLP project/models/tokenizer.pickle', 'rb')
tokenizer = pickle.load(f)
model =tf.keras.models.load_model('D:/NLP project/models/LSTMFinal Model')
padding_type='post'
sentence = ["when i go to the school i see a bitch which get fucked by an old man "]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
x=model.predict(padded)
print(x)
if x>0.5:
    print("apprpriate context")
else:
    print("inappropriate context")


[[0.9999994]]
apprpriate context
